<a href="https://colab.research.google.com/github/mbaliu-treino/Desenvolve/blob/main/LEARN_C_DS_Sistema_de_Recomendacao_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color=orange><b>Machine Learning: intro a sistemas de recomendação em Python</b></font>

<font color=gray size=2><a href="https://colab.research.google.com/drive/1GmuhJfaGYBLscfKt5arSnNWBJ4U63vgJ">Arquivo Colab</a></font>
<!--- Para indicar o arquivo a ser trabalhado a partir do Github, sem ter que criar uma nova cópias --->


<ul><font size=2 color=gray><b>FICHA TÉCNICA</b>
<li><a href="https://cursos.alura.com.br/course/introducao-a-sistemas-de-recomendacao-com-python"><font size=2 color=gray>Machine Learning: intro a sistemas de recomendação em Python</a>
<li>Carga Horária: 8 h
<li>Instrutora: Guilherme Silveira
<li>Data de Início: 10-2022
</ul>


<hr color=gray><br>

<!-- DESCRIÇÃO DOS TÓPICOS APRENDIDOS -->
<h3><b>Conteúdo / Aprendizagem:</b></h3>

- Pensamento de heurísticas e algoritmos
    - Sistemas baseados em popularidade e melhor nota,
    - Adição de filtros
    - Desvantangens nestes tipos de recomendações
- Recomendações baseadas em similaridade de usuários
- Recomendação baseado em modelos de itens e usuários
- Sistemas baseados em filtros colaborativos ou por conteúdo
- Desafios de recomendação
- Implementação de K-Nearest-Neighbours do zero
<br>

* Utilizar o for de maneira mais curta;
* Utilizar a função iloc para acessar as linhas;
* Localizar baseado no índice com set_index(‘usuarioid).loc[usuarios similares];
* Fazer uma função para ver quais filmes estão mais próximos da escolha das pessoas.
* Filtrar as distâncias;
* Extrair os usuários mais próximos;
* Analisar um subconjunto;
* Fazer uma filtragem com base nas quantidades de notas;
* O que é o KNN.
* Implementar o algoritmo KNN.
* Fazer um recomendador de filmes;
* Pegar os 10 ‘vizinhos’ mais próximos para a recomendação;

# <font color=orange>RESUMO</font>
<!-- DESCRIÇÃO DO PROCESSO DE PROCESAMENTO -->

<a href=https://docs.google.com/spreadsheets/d/1tTygYlq9r7nkUsw9a25N5_z57-de_59tSBCIVPw6KUw><font size=2 color=gray>ROTEIROS Data Science</font></a><!-- Planilha de roteiros para Data Science -->

* <font color=orange><b>



# <font color=orange>**CURSO**</font>
<!-- Descrição do projeto a ser realizado -->

**Projeto**: o projeto aqui desenvolvido é de um **sistrema de recomendação** baseado em similaridade de usuários através da análise de itens dos usuários. O case é de recomedação de filmes.


<!-- Material final oferecido no curso -->
<a href="https://github.com/alura-cursos/introducao-a-sistemas-de-recomendacao-com-python/tree/aula06"><font size=2; color=gray>Material de referencial do curso</a></font>


## Sistemas de Recomendação

O problema de recomendação pode ser resolvido de diversas maneiras.

* Pelo perfil do usuário: é possível construir modelos de perfil dos usuários (no caso de filmes, pode ser o último filme assistido, ou, as notas dadas nos filmes assitidos e identificado uma distância entre os usuário).

## 0 - Ambiente

Podemos criar ambientes virtuais para que os projetos não sejam impactados pela quantidade de bibliotecas instaladas.

In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

print('numpy:', np.__version__)
print('pandas:', pd.__version__)

numpy: 1.21.6
pandas: 1.3.5


## 1 - Dados

Os dados usuados para o sistema de recomendação é da base de dados MovieLens, que possui duas bases de dados:

1. PEQUENO: 
    - 100.000 registros de avaliações
    - 3.600 tags aplciados
    - 9.000 filmes
    - 600 usuários
    - 1 MB
2. COMPLETO:
    - 27.000.000 registros de avaliações
    - 1.100.000 tags aplciados
    - 58.000 filmes
    - 280.000 usuários
    - 265 MB

(Para a base completa, podemos usar o Spark para avaliar se há ganho de performance.)

In [47]:
url_movielens_small = 'https://files.grouplens.org/datasets/movielens/ml-latest-small.zip'
url_movielens_full = 'https://files.grouplens.org/datasets/movielens/ml-latest.zip'


dataset_size = 'small'  # small, ''

if dataset_size:
    !wget $url_movielens_small
    !unzip "/content/ml-latest-small.zip"
else:
    !wget $url_movielens_full
    !unzip "/content/ml-latest.zip"

--2022-11-01 12:41:54--  https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘ml-latest-small.zip’

ml-latest-small.zip 100%[===================>] 955.28K  3.05MB/s    in 0.3s    

2022-11-01 12:41:54 (3.05 MB/s) - ‘ml-latest-small.zip’ saved [978202/978202]

Archive:  /content/ml-latest-small.zip
   creating: ml-latest-small/
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  


In [48]:
# Filmes
filmes = pd.read_csv('/content/ml-latest-small/movies.csv')
filmes.columns = ['filmeId', 'titulo', 'generos']
filmes = filmes.set_index('filmeId')
filmes.head()

,titulo,generos
filmeId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy


In [50]:
# Notas
notas = pd.read_csv('/content/ml-latest-small/ratings.csv')
notas.columns = ['usuarioId', 'filmeId', 'nota', 'momento']
notas.head()

,usuarioId,filmeId,nota,momento
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [51]:
notas.describe().round(1)

,usuarioId,filmeId,nota,momento
count,100836.0,100836.0,100836.0,1.008360e+05
mean,326.1,19435.3,3.5,1.205946e+09
std,182.6,35531.0,1.0,2.162610e+08
min,1.0,1.0,0.5,8.281246e+08
25%,177.0,1199.0,3.0,1.019124e+09
50%,325.0,2991.0,3.5,1.186087e+09
75%,477.0,8122.0,4.0,1.435994e+09
max,610.0,193609.0,5.0,1.537799e+09


## 2 - Heuristica da Recomedação e o princípio da colaboração: baseada nas propriedades dos filmes

Quando se deseja recomendar algo, surge a pergunta: Como podemos recomendar algo? 

Ou seja, tendo vários filmes e várias notas, qual filme que recomendaremos? Logo, temos pessoas que assitiram filmes e deram notas. Cada filme teve uma quantidade diferente de visualizações. As notas representam os votos das pessoas.

(1) Neste problema, temos primeiro uma questão de AMOSTRAGEM. Nela assumiremos a premissa que estas notas são representações razoáveis do mundo real. Ou seja, que esta **amostra é representativa**.

(2) Outro aspecto da recomendação é qual tipo de informação é possuímos para realizar a recomendação. Usando somente os registros de notas, não temos conhecimento dos usuários, mas temos a popularidade do filme e o que as pessoas em geral acharam dele. Ou seja, os outros usuários da plataforma colaboraram votando nos filmes.

- Popularidade;
- Melhor nota


### 2.1 - Mais votados

In [52]:
# Mais votados
"""Tendo dois conjuntos indexados é possível usar a atribuição 
para realizar o join. O Pandas irá usar o index como chave de conexão"""

mais_votados = notas['filmeId'].value_counts()
filmes['total_votos'] = mais_votados
filmes

,titulo,generos,total_votos
filmeId,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215.0
2,Jumanji (1995),Adventure|Children|Fantasy,110.0
3,Grumpier Old Men (1995),Comedy|Romance,52.0
4,Waiting to Exhale (1995),Comedy|Drama|Romance,7.0
5,Father of the Bride Part II (1995),Comedy,49.0
...,...,...,...
193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,1.0
193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,1.0
193585,Flint (2017),Drama,1.0


In [53]:
# 5 filmes mais votados
filmes.sort_values('total_votos', ascending=False).head(10)

,titulo,generos,total_votos
filmeId,,,
356,Forrest Gump (1994),Comedy|Drama|Romance|War,329.0
318,"Shawshank Redemption, The (1994)",Crime|Drama,317.0
296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,307.0
593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller,279.0
2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,278.0
260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,251.0
480,Jurassic Park (1993),Action|Adventure|Sci-Fi|Thriller,238.0
110,Braveheart (1995),Action|Drama|War,237.0
589,Terminator 2: Judgment Day (1991),Action|Sci-Fi,224.0


### 2.2 - Maior nota

In [54]:
# + nota média dos filmes
nota_media = notas.groupby('filmeId')['nota'].mean()
filmes['nota_media'] = nota_media

filmes.sort_values('total_votos', ascending=False).head(10)

,titulo,generos,total_votos,nota_media
filmeId,,,,
356,Forrest Gump (1994),Comedy|Drama|Romance|War,329.0,4.164134
318,"Shawshank Redemption, The (1994)",Crime|Drama,317.0,4.429022
296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,307.0,4.197068
593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller,279.0,4.161290
2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,278.0,4.192446
260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,251.0,4.231076
480,Jurassic Park (1993),Action|Adventure|Sci-Fi|Thriller,238.0,3.750000
110,Braveheart (1995),Action|Drama|War,237.0,4.031646
589,Terminator 2: Judgment Day (1991),Action|Sci-Fi,224.0,3.970982


In [55]:
# 10 filmes com maior nota
filmes.sort_values('nota_media', ascending=False).head(10)

,titulo,generos,total_votos,nota_media
filmeId,,,,
88448,Paper Birds (Pájaros de papel) (2010),Comedy|Drama,1.0,5.0
100556,"Act of Killing, The (2012)",Documentary,1.0,5.0
143031,Jump In! (2007),Comedy|Drama|Romance,1.0,5.0
143511,Human (2015),Documentary,1.0,5.0
143559,L.A. Slasher (2015),Comedy|Crime|Fantasy,1.0,5.0
6201,Lady Jane (1986),Drama|Romance,1.0,5.0
102217,Bill Hicks: Revelations (1993),Comedy,1.0,5.0
102084,Justice League: Doom (2012),Action|Animation|Fantasy,1.0,5.0
6192,Open Hearts (Elsker dig for evigt) (2002),Romance,1.0,5.0


### 2.3 - Efeito do nicho (enviesamento de representatividade)

**Dificuldades da heurística univariada**: A recomendação baseado em somente uma única coluna pode apresentar diversas falhas substanciais no objetivo da recomendação. Na seleção dos filmes mais populares, primeiro não definimos o que seria a popularidade e, segundo, alguns filmes com notas mais baixas foram selecionados, já que diversas pessoas votaram nele, mas não necessariamente acharam ele bom.

O segundo experimento foi usado a nota como elemento para a seleção. No entanto, percebemos que filmes com poucas notas (todos só tinham um voto) foram os que tiveram as melhores notas, já que possuem alto grau de viés. Neste tipo de seleção encontramos o problema dos nichos, filmes que são assistidos por poucas pessoas. Neste caso, os filmes provavelmente não irão interessas à maioria das pessoas.

Com isso, encontramos um dilema da heurística da recomendação. O simples rankeamento não é suficiente. O que podemos fazer é realizar filtros, usar *threshold*, para limitar algumas situações. 

Mas nestes casos teremos que tomar uma decisão arbitrária, para obter resultados melhores. Podemos ajustar estes filtros com a reação dos clientes. Se for satisfatório, o sistema de recomendação estará execendo a função dele, mas se os clientes não estiverem gostando do sistema, deveremos alterar os filtros.

In [56]:
# Filtro 
seletor = filmes['total_votos'] > 50
filmes[seletor].sort_values('nota_media', ascending=False).head(10)

,titulo,generos,total_votos,nota_media
filmeId,,,,
318,"Shawshank Redemption, The (1994)",Crime|Drama,317.0,4.429022
858,"Godfather, The (1972)",Crime|Drama,192.0,4.289062
2959,Fight Club (1999),Action|Crime|Drama|Thriller,218.0,4.272936
1276,Cool Hand Luke (1967),Drama,57.0,4.271930
750,Dr. Strangelove or: How I Learned to Stop Worr...,Comedy|War,97.0,4.268041
904,Rear Window (1954),Mystery|Thriller,84.0,4.261905
1221,"Godfather: Part II, The (1974)",Crime|Drama,129.0,4.259690
48516,"Departed, The (2006)",Crime|Drama|Thriller,107.0,4.252336
1213,Goodfellas (1990),Crime|Drama,126.0,4.250000


## 3 - Heurística através do Usuário

Para melhorar a qualidade das recomendações precisamos é ter mais critérios para a seleção. Uma forma trazer novos critérios para a realziação de recomendações é acrescentando informações dos **usuários** para o sistema.

Com isso, pode-se analisar o histórico de filmes assistidos pelo usuário e aplicar no sistema de recomendação. Com isso, tem-se uma recomendação mais personalizada.

Mas algumas dilemas ainda persistem, tais como se filmes de nicho devem ou não ser mostrados e como identificá-los.


In [57]:
# Exemplo de situação
eu_assisti = [1, 21, 19, 10, 11, 7, 2]
filmes.loc[eu_assisti]

,titulo,generos,total_votos,nota_media
filmeId,,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215.0,3.920930
21,Get Shorty (1995),Comedy|Crime|Thriller,89.0,3.494382
19,Ace Ventura: When Nature Calls (1995),Comedy,88.0,2.727273
10,GoldenEye (1995),Action|Adventure|Thriller,132.0,3.496212
11,"American President, The (1995)",Comedy|Drama|Romance,70.0,3.671429
7,Sabrina (1995),Comedy|Romance,54.0,3.185185
2,Jumanji (1995),Adventure|Children|Fantasy,110.0,3.431818


O último filme visto era do gênero aventura, infantil e fantasia. Qual outro filme semelhante que pode ser recomendado?

In [58]:
# Seleção de filmes bem votados do mesmo genero
df = filmes.query('generos == "Adventure|Children|Fantasy"')
df = df.sort_values('nota_media', ascending=False)
df.head()

,titulo,generos,total_votos,nota_media
filmeId,,,,
126142,The Cave of the Golden Rose (1991),Adventure|Children|Fantasy,1.0,4.000000
80748,Alice in Wonderland (1933),Adventure|Children|Fantasy,1.0,4.000000
56915,"Water Horse: Legend of the Deep, The (2007)",Adventure|Children|Fantasy,1.0,4.000000
82169,Chronicles of Narnia: The Voyage of the Dawn T...,Adventure|Children|Fantasy,7.0,3.857143
4896,Harry Potter and the Sorcerer's Stone (a.k.a. ...,Adventure|Children|Fantasy,107.0,3.761682


In [59]:
# Seleção de filmes bem votados do mesmo genero
"""Pode-se realizar a mesma operação heurística para remoção de filmes de nicho
O resultado traz alguns filmes de nicho e outros populares"""

df = filmes.query('total_votos > 50')
df = df.query('generos == "Adventure|Children|Fantasy"')
df = df.sort_values('nota_media', ascending=False)
# Remoção dos filmes já vistos
df = df.drop(eu_assisti, errors='ignore')
df.head()

,titulo,generos,total_votos,nota_media
filmeId,,,,
4896,Harry Potter and the Sorcerer's Stone (a.k.a. ...,Adventure|Children|Fantasy,107.0,3.761682
41566,"Chronicles of Narnia: The Lion, the Witch and ...",Adventure|Children|Fantasy,62.0,3.443548


In [89]:
# Seleção de filmes bem votados do mesmo genero
"""Pode-se realizar a mesma operação heurística para remoção de filmes de nicho,
mas baseado no nível de nicho (popularidade) do filme assistido"""

pop_filme = 100  # popularidade do filme Jumanji 
pop_filme *= 0.25

df = filmes.query(f'total_votos > {pop_filme}')
df = df.query('generos == "Adventure|Children|Fantasy"')
df = df.sort_values('nota_media', ascending=False)
df = df.drop(eu_assisti, errors='ignore')
df.head()

,titulo,generos,total_votos,nota_media
filmeId,,,,
4896,Harry Potter and the Sorcerer's Stone (a.k.a. ...,Adventure|Children|Fantasy,107.0,3.761682
2161,"NeverEnding Story, The (1984)",Adventure|Children|Fantasy,43.0,3.581395
41566,"Chronicles of Narnia: The Lion, the Witch and ...",Adventure|Children|Fantasy,62.0,3.443548
60,"Indian in the Cupboard, The (1995)",Adventure|Children|Fantasy,34.0,3.235294


## 4 - Modelo baseado em distância entre usuários

Existem várias possibilidades para o <font color=orange><b>problema de recomendação</b></font>, como sendo um problema de regressão (com as probabilidades), de classificação (qual elemento a recomendar), de ranking (onde está o filme a ser recomendado).

O novo modelo aqui criado usa a nota do usuário ao filmes para identificar o seu <font color=orange><b>PERFIL</b></font>. 

O conceito de PERFIL já foi usado anteriormente quando considerado o último filme assistido pelo usuário. Isso já é um modelo simples do usuário.

Nesse novo modelo de perfil é considerado a nota para identificar outros <font color=orange><b>usuários similares</b></font>.

<font color=orange><b>DISTÂNCIA DE SIMILARIDADE</b></font>: Uma forma de calcular a similaridade entre duas entidades, neste caso usuários, pode-se calcular a distância entre eles. Usando os atributos numéricos dos usuários é possível alguma medida de distância entre eles, que representaria a distância entre seus perfis. 

Um algorítmo que utiliza este mesmo princípio é o KNN.

O novo modelo de recomendação irá utilizar utilizar o princípio da **sistema colaborativo**, em que os próprios usuários atribuem notas para os filmes. Com isso será possível criar um perfil de cada usuário para identificar outros usuários similares e conseguir recomendar filmes que eles gostaram.

### 4.1 - Distância entre dois usuários

Através da comparação entre os mesmos filmes votados em uma dupla de usuários é obtida a distância euclidiana entre eles. Isto poderá ser **abstraído** e **generalizado** para ser usada como uma medida da distância entre o gosto desses dois usuários.

In [61]:
# Método de pitágoras no numpy
np.linalg.norm([3,4])

5.0

In [62]:
# Função de distância
def distancia_vetores(a, b):
    return np.linalg.norm(a-b)

In [63]:
# Usuário 1: notas
notas_do_usuario = notas.query('usuarioId == 1')[['filmeId', 'nota']].set_index('filmeId')
notas_do_usuario

,nota
filmeId,
1,4.0
3,4.0
6,4.0
47,5.0
50,5.0
...,...
3744,4.0
3793,5.0
3809,4.0


In [64]:
# Função de seleção das notas de usuário
def notas_do_usuario(usuario):
    notas_usuario = notas.query(f'usuarioId == {usuario}')[['filmeId', 'nota']].set_index('filmeId')
    return notas_usuario

In [65]:
notas_do_usuario(1)

,nota
filmeId,
1,4.0
3,4.0
6,4.0
47,5.0
50,5.0
...,...
3744,4.0
3793,5.0
3809,4.0


In [66]:
user1 = notas_do_usuario(1)
user4 = notas_do_usuario(4)

# LEFT JOIN
# diferenca = user1.join(user4, lsuffix='_l', rsuffix='_r').dropna()

diferenca = pd.merge(user1, user4, 
         left_index=True, right_index=True, how='inner')

diferenca

,nota_x,nota_y
filmeId,,
47,5.0,2.0
235,4.0,2.0
260,5.0,5.0
296,3.0,1.0
441,4.0,1.0
457,5.0,5.0
553,5.0,2.0
593,4.0,5.0
608,5.0,5.0


In [67]:
distancia_vetores(diferenca['nota_x'], diferenca['nota_y'])

11.135528725660043

In [68]:
def distancia_usuarios(usuario_id1, usuario_id2):
    notas1 = notas_do_usuario(usuario_id1)
    notas2 = notas_do_usuario(usuario_id2)
    diferenca = notas1.join(notas2, lsuffix='_l', rsuffix='_r').dropna()
    distancia = distancia_vetores(diferenca['nota_l'], diferenca['nota_r'])
    return [usuario_id1, usuario_id2, distancia]

distancia_usuarios(1,4)

[1, 4, 11.135528725660043]

### 4.2 - Distância entre um e todos

Anteriormente calculamos a distância entre dois usuários. Agora é preciso saber a distância de um usuário com todos os outros, para saber com quais ele é mais semelhante e a quais ele é mais distante.

In [69]:
# Conjunto de usuários
quantidade_usuarios = notas['usuarioId'].unique()
print('Total de usuários no sistema:', len(quantidade_usuarios))

Total de usuários no sistema: 610


In [70]:
# Distância entre um usuário e todos os outros [forma imperativa]

def distancia_de_todos(usuario_origem):
    distancias = []
    for usuario_id in quantidade_usuarios[:25]:
        informacoes = distancia_usuarios(usuario_origem, usuario_id)
        distancias.append(informacoes)

    return distancias

distancia_de_todos(1)

[[1, 1, 0.0],
 [1, 2, 1.4142135623730951],
 [1, 3, 8.200609733428363],
 [1, 4, 11.135528725660043],
 [1, 5, 3.7416573867739413],
 [1, 6, 8.602325267042627],
 [1, 7, 7.106335201775948],
 [1, 8, 3.7416573867739413],
 [1, 9, 1.0],
 [1, 10, 7.858116822750856],
 [1, 11, 4.242640687119285],
 [1, 12, 0.7071067811865476],
 [1, 13, 1.4142135623730951],
 [1, 14, 5.0],
 [1, 15, 5.852349955359813],
 [1, 16, 7.22841614740048],
 [1, 17, 4.6097722286464435],
 [1, 18, 8.426149773176359],
 [1, 19, 17.60681686165901],
 [1, 20, 6.48074069840786],
 [1, 21, 9.096702699330127],
 [1, 22, 5.244044240850758],
 [1, 23, 5.408326913195984],
 [1, 24, 4.415880433163924],
 [1, 25, 1.4142135623730951]]

In [71]:
# Distância entre um usuário e todos os outros [list comprehensive]
def distancia_de_todos(usuario_origem):
    quantidade_usuarios = notas['usuarioId'].unique()
    todos_usuarios = quantidade_usuarios[:300]
    distancias = (distancia_usuarios(1, usuario_id) for usuario_id in todos_usuarios)
    distancias = pd.DataFrame(distancias, columns=['origem', 'outra_pessoa', 'distancia'])
    return distancias

distancia_de_todos(1)

,origem,outra_pessoa,distancia
0,1,1,0.000000
1,1,2,1.414214
2,1,3,8.200610
3,1,4,11.135529
4,1,5,3.741657
...,...,...,...
295,1,296,3.741657
296,1,297,4.898979
297,1,298,13.095801
298,1,299,3.000000


Ao avaliar este resultado, percebe-se que há mutas notas 0. Provavelmente são eventos em que não há filmes assistido entre os dois.

In [72]:
# Teste de comparação entre notas dadas por dois usuários
notas_do_usuario(1).join(notas_do_usuario(5), lsuffix='_1').dropna()

,nota_1,nota
filmeId,,
1,4.0,4.0
50,5.0,4.0
110,4.0,4.0
296,3.0,5.0
316,3.0,2.0
349,4.0,3.0
367,4.0,4.0
457,5.0,4.0
527,5.0,5.0


### 4.3 - Ajuste da função de distância

Devido aos casos de poucas notas ou até mesmo nenhuma nota prejudicar o conceito de similaridade, iremos ajustar a função da distância para limitar a quantidade mpinima de filmes avaliada por dosi usuários.

In [73]:
def distancia_usuarios(usuario_id1, usuario_id2):
    notas1 = notas_do_usuario(usuario_id1)
    notas2 = notas_do_usuario(usuario_id2)
    diferencas = notas1.join(notas2, lsuffix='_l', rsuffix='_r').dropna()
    minimo = 5
    if len(diferencas) < minimo:
        return [usuario_id1, usuario_id2, 100000]

    distancia = distancia_vetores(diferencas['nota_l'], diferencas['nota_r'])
    return [usuario_id1, usuario_id2, distancia]

distancia_usuarios(1,4)

[1, 4, 11.135528725660043]

In [74]:
distancia_de_todos(1)

,origem,outra_pessoa,distancia
0,1,1,0.000000
1,1,2,100000.000000
2,1,3,8.200610
3,1,4,11.135529
4,1,5,3.741657
...,...,...,...
295,1,296,3.741657
296,1,297,4.898979
297,1,298,13.095801
298,1,299,3.000000


### 4.4 - Ordenando usuários por distância

In [75]:
def mais_proximos(usuario_origem):
    distancias = distancia_de_todos(usuario_origem)
    distancias = distancias.sort_values('distancia', ascending=True)
    distancias = distancias.set_index('outra_pessoa').drop(usuario_origem)
    return distancias

mais_proximos(1)

,origem,distancia
outra_pessoa,,
77,1,0.000000
258,1,1.000000
49,1,1.000000
9,1,1.000000
65,1,1.322876
...,...,...
148,1,100000.000000
253,1,100000.000000
278,1,100000.000000


### 4.5 - Refatoração para teste

O algorítmo não possui uma grande performance, então iremos limitar a quantidade de elementos comparados para acelerar as execuções de testes.

In [76]:
# Distância entre um usuário e todos os outros [list comprehensive]
def distancia_de_todos(usuario_origem, amostra=None):
    todos_usuarios = notas['usuarioId'].unique()
    if amostra:
        todos_usuarios = todos_usuarios[:amostra]
    distancias = (distancia_usuarios(1, usuario_id) for usuario_id in todos_usuarios)
    distancias = pd.DataFrame(distancias, columns=['origem', 'outra_pessoa', 'distancia'])
    return distancias

distancia_de_todos(1, amostra=300)

,origem,outra_pessoa,distancia
0,1,1,0.000000
1,1,2,100000.000000
2,1,3,8.200610
3,1,4,11.135529
4,1,5,3.741657
...,...,...,...
295,1,296,3.741657
296,1,297,4.898979
297,1,298,13.095801
298,1,299,3.000000


In [77]:
def mais_proximos(usuario_origem, amostra=None):
    distancias = distancia_de_todos(usuario_origem, amostra=amostra)
    distancias = distancias.sort_values('distancia', ascending=True)
    distancias = distancias.set_index('outra_pessoa').drop(usuario_origem)
    return distancias

mais_proximos(1, amostra=50)

,origem,distancia
outra_pessoa,,
49,1,1.000000
9,1,1.000000
13,1,1.414214
25,1,1.414214
30,1,1.802776
35,1,2.236068
26,1,2.236068
46,1,3.316625
8,1,3.741657


### 4.6 - Limpeza das distâncias excessivas

Para remover os casos em que não se aplica, podemos removê-los.

In [78]:
def distancia_usuarios(usuario_id1, usuario_id2):
    notas1 = notas_do_usuario(usuario_id1)
    notas2 = notas_do_usuario(usuario_id2)
    diferencas = notas1.join(notas2, lsuffix='_l', rsuffix='_r').dropna()
    minimo = 5
    if len(diferencas) < minimo:
        return None

    distancia = distancia_vetores(diferencas['nota_l'], diferencas['nota_r'])
    return [usuario_id1, usuario_id2, distancia]

distancia_usuarios(1,4)

[1, 4, 11.135528725660043]

A função `filter` retorna os dados que não forem compatíveis com o primeiro argumento de comparação. Outro aspecto importante é que ele não retona uma lista, mas sim um objeto filter, como um *generator*.

In [79]:
# Distância entre um usuário e todos os outros [list comprehensive]
def distancia_de_todos(usuario_origem, amostra=None):
    todos_usuarios = notas['usuarioId'].unique()
    if amostra:
        todos_usuarios = todos_usuarios[:amostra]
    distancias = (distancia_usuarios(1, usuario_id) for usuario_id in todos_usuarios)
    distancias = filter(None, distancias)
    distancias = pd.DataFrame(distancias, columns=['origem', 'outra_pessoa', 'distancia'])
    return distancias


mais_proximos(1, amostra=100)

,origem,distancia
outra_pessoa,,
77,1,0.000000
9,1,1.000000
49,1,1.000000
65,1,1.322876
90,1,1.414214
...,...,...
57,1,13.190906
91,1,13.518506
28,1,14.882876


## 5 - Implementação da recomendação baseado no usuário

### 5.1 - Baseado em um único usuário

In [80]:
# Identificação do usuário mais similar
usuario_chave = 1
similares = mais_proximos(usuario_chave, amostra=50)

usuario_mais_similar = similares.iloc[0].name

In [81]:
def sugestao(usuario_chave, amostra=None):
    # Notas do usuário mais similar
    similares = mais_proximos(usuario_chave, amostra=amostra)
    usuario_mais_similar = similares.iloc[0].name
    notas_do_similar = notas_do_usuario(usuario_mais_similar)

    # Remoção dos filmes já assistidos
    filmes_ja_assitidos = notas_do_usuario(usuario_chave).index
    notas_do_similar = notas_do_similar.drop(filmes_ja_assitidos, errors='ignore')
    recomendacoes = notas_do_similar.sort_values('nota', ascending=False)
    return recomendacoes.head(10).join(filmes)

sugestao(1, amostra=50)

,nota,titulo,generos,total_votos,nota_media
filmeId,,,,,
1200,4.5,Aliens (1986),Action|Adventure|Horror|Sci-Fi,126.0,3.964286
4022,4.5,Cast Away (2000),Drama,100.0,3.700000
47099,4.5,"Pursuit of Happyness, The (2006)",Drama,46.0,3.793478
79132,4.5,Inception (2010),Action|Crime|Drama|Mystery|Sci-Fi|Thriller|IMAX,143.0,4.066434
109487,4.5,Interstellar (2014),Sci-Fi|IMAX,73.0,3.993151
139385,4.5,The Revenant (2015),Adventure|Drama,31.0,3.903226
168252,4.5,Logan (2017),Action|Sci-Fi,25.0,4.280000
318,4.0,"Shawshank Redemption, The (1994)",Crime|Drama,317.0,4.429022
5218,4.0,Ice Age (2002),Adventure|Animation|Children|Comedy,85.0,3.688235


### 5.2 - Baseado em um conjunto de usuários (KNN)

Ao invés de se basear em um único elemento, neste caso, um usuário, podemos usar um conjunto de usuários mais similares para identificar qual o gosto genérico deles para recomendar a aum usuário. Então é a sugestão baseada em k indivíduos mais próximos, KNN.

Com isso, busca-se obter maior generalização e reduzir o impacto da idiossincrasia de poucos usuários.

In [99]:
# Adicição do número do tamanho do conjunto dos mais próximos
def mais_proximos(usuario_origem, k_mais_proximos=10, amostra=None):
    distancias = distancia_de_todos(usuario_origem, amostra=amostra)
    distancias = distancias.sort_values('distancia', ascending=True)
    distancias = distancias.set_index('outra_pessoa').drop(usuario_origem, errors='ignore')
    return distancias.head(k_mais_proximos)

mais_proximos(1, amostra=50)

In [83]:
def sugestao_knn(usuario_chave, k_mais_proximos=10, amostra=None):
    # Notas do usuário mais similar
    similares = mais_proximos(usuario_chave, k_mais_proximos=k_mais_proximos, amostra=amostra)
    usuario_mais_similar = similares.index
    notas_dos_similares = notas.set_index('usuarioId').loc[usuario_mais_similar]
    recomendacoes = notas_dos_similares.groupby('filmeId')[['nota']].mean()
    recomendacoes = recomendacoes.sort_values('nota', ascending=False)
    return recomendacoes.head(10).join(filmes)


usuario_chave=1
k_mais_proximos = 10

numero_de_usuarios_a_analisar = 50
sugestao_knn(usuario_chave, k_mais_proximos=k_mais_proximos, amostra=numero_de_usuarios_a_analisar)

,nota,titulo,generos,total_votos,nota_media
filmeId,,,,,
187593,5.0,Deadpool 2 (2018),Action|Comedy|Sci-Fi,12.0,3.875000
68157,5.0,Inglourious Basterds (2009),Action|Drama|War,88.0,4.136364
475,5.0,In the Name of the Father (1993),Drama,25.0,4.300000
33794,5.0,Batman Begins (2005),Action|Crime|IMAX,116.0,3.862069
7153,5.0,"Lord of the Rings: The Return of the King, The...",Action|Adventure|Drama|Fantasy,185.0,4.118919
5952,5.0,"Lord of the Rings: The Two Towers, The (2002)",Adventure|Fantasy,188.0,4.021277
5902,5.0,Adaptation (2002),Comedy|Drama|Romance,46.0,3.945652
247,5.0,Heavenly Creatures (1994),Crime|Drama,21.0,3.928571
5481,5.0,Austin Powers in Goldmember (2002),Comedy,65.0,2.846154


### 5.3 - Generalização do Algorítmo

O algorítmo criado usa os dados de um usuário presente na base de usuários já existentes. Logo, não permite a especificação de um usuário novo.

In [84]:
def novo_usuario(dados):
    novo_usuario_id = notas['usuarioId'].max()+1
    notas_do_usuario_novo  = pd.DataFrame(dados, columns=["filmeId", "nota"])
    notas_do_usuario_novo['usuarioId'] = novo_usuario_id
    return pd.concat([notas, notas_do_usuario_novo])

In [85]:
notas = novo_usuario([[122904,2],[1246,5],[2529,2],[2329,5],[2324,5],[1,2],[7,0.5],[2,2],[1196,1],[260,1]])
notas.tail(12)

,usuarioId,filmeId,nota,momento
100834,610,168252,5.0,1.493846e+09
100835,610,170875,3.0,1.493846e+09
0,611,122904,2.0,NaN
1,611,1246,5.0,NaN
2,611,2529,2.0,NaN
3,611,2329,5.0,NaN
4,611,2324,5.0,NaN
5,611,1,2.0,NaN
6,611,7,0.5,NaN
7,611,2,2.0,NaN


In [88]:
sugestao_knn(611, k_mais_proximos=10, amostra=None)

"""Temos recomendações com pouca quantidade de votos, trazendo filmes de nicho.
Talvez fosse interessante remover este viés de representatividade"""

,nota,titulo,generos,total_votos,nota_media
filmeId,,,,,
55167,5.0,Tekkonkinkreet (Tekkon kinkurîto) (2006),Action|Adventure|Animation|Crime|Fantasy,4.0,4.625000
8970,5.0,Finding Neverland (2004),Drama,32.0,3.609375
37729,5.0,Corpse Bride (2005),Animation|Comedy|Fantasy|Musical|Romance,44.0,3.534091
2005,5.0,"Goonies, The (1985)",Action|Adventure|Children|Comedy|Fantasy,57.0,3.570175
31696,5.0,Constantine (2005),Action|Fantasy|Horror|Thriller,35.0,3.471429
2018,5.0,Bambi (1942),Animation|Children|Drama,38.0,3.368421
2033,5.0,"Black Cauldron, The (1985)",Adventure|Animation|Children|Fantasy,8.0,3.750000
2048,5.0,"Great Mouse Detective, The (1986)",Action|Animation|Children|Crime,8.0,3.562500
2058,5.0,"Negotiator, The (1998)",Action|Crime|Drama|Mystery|Thriller,30.0,3.350000


In [94]:
filmes_com_mais_de_50_votos = filmes.query('total_votos > 50')
notas = notas.set_index("filmeId").loc[filmes_com_mais_de_50_votos.index]
notas = notas.reset_index()

sugestao_knn(611, k_mais_proximos=10, amostra=None)

,nota,titulo,generos,total_votos,nota_media
filmeId,,,,,
2115,5.0,Indiana Jones and the Temple of Doom (1984),Action|Adventure|Fantasy,108.0,3.638889
1220,5.0,"Blues Brothers, The (1980)",Action|Comedy|Musical,84.0,3.809524
1517,5.0,Austin Powers: International Man of Mystery (1...,Action|Adventure|Comedy,100.0,3.535000
3996,5.0,"Crouching Tiger, Hidden Dragon (Wo hu cang lon...",Action|Drama|Romance,110.0,3.836364
1291,5.0,Indiana Jones and the Last Crusade (1989),Action|Adventure,140.0,4.046429
1282,5.0,Fantasia (1940),Animation|Children|Fantasy|Musical,53.0,3.783019
1278,5.0,Young Frankenstein (1974),Comedy|Fantasy,69.0,3.992754
1270,5.0,Back to the Future (1985),Adventure|Comedy|Sci-Fi,171.0,4.038012
4995,5.0,"Beautiful Mind, A (2001)",Drama|Romance,123.0,4.000000


### 5.4 - Filmes representativos dos K usuários

In [114]:
def sugestao_knn(usuario_chave, k_mais_proximos=10, amostra=None):
    # Notas do usuário mais similar
    similares = mais_proximos(usuario_chave, k_mais_proximos=k_mais_proximos, amostra=amostra)
    usuario_mais_similar = similares.index
    notas_dos_similares = notas.set_index('usuarioId').loc[usuario_mais_similar]
    recomendacoes = notas_dos_similares.groupby('filmeId')[['nota']].agg(['mean', 'count'])
    filtro_minimo = k_mais_proximos/2
    selector = recomendacoes['nota']['count'] > 5
    recomendacoes = recomendacoes[selector]
    recomendacoes = recomendacoes.sort_values(('nota', 'mean'), ascending=False)

    return recomendacoes.head(10).join(filmes)


usuario_chave=1
k_mais_proximos = 10

numero_de_usuarios_a_analisar = None
resp = sugestao_knn(usuario_chave, k_mais_proximos=k_mais_proximos, amostra=numero_de_usuarios_a_analisar)
resp

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:9138: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left,1 on the right)
  sort=sort,


,"(nota, mean)","(nota, count)",titulo,generos,total_votos,nota_media
filmeId,,,,,,
2571,4.916667,6,"Matrix, The (1999)",Action|Sci-Fi|Thriller,278.0,4.192446
7153,4.750000,6,"Lord of the Rings: The Return of the King, The...",Action|Adventure|Drama|Fantasy,185.0,4.118919
58559,4.750000,6,"Dark Knight, The (2008)",Action|Crime|Drama|IMAX,149.0,4.238255
4993,4.714286,7,"Lord of the Rings: The Fellowship of the Ring,...",Adventure|Fantasy,198.0,4.106061
318,4.583333,6,"Shawshank Redemption, The (1994)",Crime|Drama,317.0,4.429022
2959,4.583333,6,Fight Club (1999),Action|Crime|Drama|Thriller,218.0,4.272936
109487,4.583333,6,Interstellar (2014),Sci-Fi|IMAX,73.0,3.993151
527,4.571429,7,Schindler's List (1993),Drama|War,220.0,4.225000
79132,4.357143,7,Inception (2010),Action|Crime|Drama|Mystery|Sci-Fi|Thriller|IMAX,143.0,4.066434


# Considerações FInais

* **DIMENSÕES VARIÁVEIS**: Neste problema, o algorítmo não possui uma quantidade de colunas fixas, já que alguns filmes só foram assistidos por uma dupla, mas não por outra. Para resolver este problema, a distância é calculada somente entre dois usuários, usando as dimensões em comum entre eles.
* **CÁLCULO DA DISTÂNCIA**: a distância pode ser calculada de diversas formas e cada uma ressalta algum aspecto ou propriedade e também possui os seus reveses [[Sobre as distâncias](https://towardsdatascience.com/9-distance-measures-in-data-science-918109d069fa)]. A distância euclidiana é boa para entender a ideia da distânica, mas a medida pode crescer de acordo com o aumento de dimensões. Não existe um limite máximo da distância. Então no modelo proposto, há no modelo uma interpretação da quantidade de notas que cada um deu.
$$\text{Máx} = d_{máx} * \sqrt{n_{dimensões}}$$
    
    * Uma alternativa é usar a distância angular, que só considera distância entre o angulo dos elementos e não a intensidade no espaço.
    * Ou a distância manhattan, que é uma soma das distâncias ortogonais. Neste caso o valor máximo distância 

**REFINAMENTO**

Como encontrar um recomendador melhor?

* Refinar o cálculo de distância:
* Refinar a forma de agrupamento dos K usuários: grupos de usuários, grupos de filmes de acordo com o gênero, 

* Surprise: recsys


Recomendações de nicho:

- Filmes que ainda não são populares,
- Filmes que estão ficando populares,
- "Descubra filmes novos" (filmes de nicho)
- ...